In [84]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [85]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from dateutil.relativedelta import relativedelta
import json

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

# Local imports
from data_fetcher import DataFetcher
# from preprocessing import Processor

In [66]:
# Define some string constants for easy typing
SAMPLE_DATA_BY_SITE = 'sampleData/bySite'
SAMPLE_DATA_BY_COUNTY = 'sampleData/byCounty'
SAMPLE_DATA_BY_STATE = 'sampleData/byState'
SAMPLE_DATA_BY_BOX = 'sampleData/byBox'
SAMPLE_DATA_BY_CBSA = 'sampleData/byCBSA'

LIST_STATES = 'list/states'
LIST_COUNTIES_BY_STATE = 'list/countiesByState'
LIST_SITES_BY_COUNTY = 'list/sitesByCounty'
LIST_CBSAs = 'list/cbsas'
LIST_PARAM_CLASSES = 'list/classes'
LIST_PARAM_IN_CLASS = 'list/parametersByClass'

In [67]:
datafetcher = DataFetcher()

In [68]:
# Example calls to list codes
cali_code = datafetcher.get_codes(LIST_STATES, all=False, value='California')
print('California state code:', cali_code)

criteria_code = datafetcher.get_codes(LIST_PARAM_CLASSES, all=False, value='Criteria Pollutants')
print('Criteria polutants code:', criteria_code)

carbon_monoxide_code = datafetcher.get_codes(LIST_PARAM_IN_CLASS, all=False, value='Carbon monoxide', nparams={'pc':criteria_code})
print('Carbon monoxide code:', carbon_monoxide_code)

California state code: 06
Criteria polutants code: CRITERIA
Carbon monoxide code: 42101


In [69]:
datafetcher.all_codes

,value_represented
code,
11101,Suspended particulate (TSP)
11102,Suspended particulate (TSP) LC
11103,Benzene soluble organics (TSP)
11104,Total polynuclear hydrocarbons
11114,Windblown particulate
...,...
88500,PM2.5 Total Atmospheric
88501,PM2.5 Raw Data
88502,Acceptable PM2.5 AQI & Speciation Mass


In [70]:
_ = datafetcher.find_code('Cristabalite', verbose=True)

Cristabalite code is: 11122


## Explore data by area codes to find good location for modelling

We want to find a site, or small grroup of sites, that have enough data for us to train a useeful model. We need this set to contain metereological, ozone, particulate matter, and pollutant data.

We sample a day per year for 5 of the last 20 years and find the site with the best data for this particular county/state pair. We do this sampling because the API will lock us out if we try to get too much yearly data.

This code takes roughly 30 minutes to run. And it finds that Los Angels-North Main Street had the most data.

In [71]:
# r = datafetcher.find_best_location()
# with open('data.json', 'w') as fp:
#     json.dump(r, fp)

with open('data.json', 'r') as f:
  r = json.load(f)

Let's find which sites have the most data

In [72]:
data = r['Data']
metadata = r['Metadata']

num_codes = len(data['Azusa']) # Any key in result dict
num_years = len(data['Azusa'][0])

yearly_results = {site:[sum([data[site][code][year] for code in range(num_codes)]) for year in range(num_years)] for site in data}
yearly_best = {site: ([date for date, val in enumerate(yearly_results[site]) if val == max(yearly_results[site])], max(yearly_results[site])) for site in yearly_results}
yearly_best_sorted = sorted(yearly_best.items(), key=lambda x : x[1][1], reverse=True)
yearly_best_sorted[:5] # NOTE: (year whree most measurements weree takeen, most mausrements taken)

[('Burbank', ([2], 13)),
 ('Los Angeles-North Main Street', ([3, 4], 13)),
 ('Azusa', ([1], 12)),
 ('Pico Rivera #2', ([2], 12)),
 ('Santa Clarita', ([2], 12))]

In [73]:
dates = [i[0]+':'+i[1] for i in metadata['dates']]
codes = [datafetcher.all_codes.loc[code]['value_represented'] for code in metadata['codes']]

ndf = pd.DataFrame(data['Los Angeles-North Main Street'])
ndf.index = codes
ndf.columns = dates
ndf # NOTE: Table for when we have values in chosen station!

,20000528:20000529,20051030:20051031,20100301:20100302,20150413:20150414,20200702:20200703
Carbon monoxide,True,True,True,True,True
Nitrogen dioxide (NO2),True,True,True,True,True
Ozone,True,True,True,True,True
PM2.5 - Local Conditions,True,True,True,True,True
Wind Direction - Resultant,True,True,False,True,True
Wind Speed - Resultant,True,True,False,True,True
Outdoor Temperature,True,True,True,True,True
Relative Humidity,True,True,True,True,True
Solar radiation,True,False,False,True,True
Ultraviolet radiation,False,False,False,True,True


From the results above we will proceed with <b>Los Angeles-North Main Street</b> as our primary location to model. Notice that it has data on all our criteria pollutants and MET variables. We now proceed to find the amount of VOC data we have for these sites with the same sampling as before.

<b>SAVE CODES AND YEARS IN DICT TOO, USE DATES TO SEARCH FOR VOC</b>

In [74]:
# Pick 5 best sites
best_sites = [i[0] for i in yearly_best_sorted[:5]]
best_sites_codes = [datafetcher.get_codes(LIST_SITES_BY_COUNTY, all=False, value=i, nparams={'state':'06', 'county':'037'}) for i in best_sites]
best_sites_dates = [[metadata['dates'][j] for j in i[1][0]] for i in yearly_best_sorted[:5]]
best_sites

['Burbank',
 'Los Angeles-North Main Street',
 'Azusa',
 'Pico Rivera #2',
 'Santa Clarita']

In [75]:
# voc_r = datafetcher.find_voc_availability(best_sites, best_sites_codes, best_sites_dates)
# with open('voc_data.json', 'w') as f:
#     json.dump(voc_r, f)

with open('voc_data.json', 'r') as f:
  voc_r = json.load(f)

In [76]:
voc_data = np.array(voc_r['Data'])
voc_df = pd.DataFrame(voc_r['Data'])
voc_df.index = voc_r['Metadata']['codes']
voc_df.head(5)

,Burbank,Los Angeles-North Main Street,Azusa,Pico Rivera #2,Santa Clarita
43000,[False],"[False, True]",[True],[False],[False]
43102,[False],"[False, True]",[True],[False],[False]
43202,[False],"[False, True]",[True],[False],[False]
43203,[False],"[False, True]",[True],[False],[False]
43204,[False],"[False, True]",[True],[False],[False]


In [77]:
voc_site_results = {}
for site in voc_r['Data']:
    arr = np.array(voc_r['Data'][site])
    voc_site_results[site] = arr.sum(axis=0)
voc_site_results

{'Burbank': array([-1]),
 'Los Angeles-North Main Street': array([ 0, 59]),
 'Azusa': array([56]),
 'Pico Rivera #2': array([0]),
 'Santa Clarita': array([0])}

In [78]:
voc_r['Metadata']['dates']

[[['20100301', '20100302']],
 [['20150413', '20150414'], ['20200702', '20200703']],
 [['20051030', '20051031']],
 [['20100301', '20100302']],
 [['20100301', '20100302']]]

### Explore dataset for chosen sight W VOC data

Los Angeles-North Main Street had the most CRITERIA, MET, and VOC data (almost all the PAMS_VOCS are in this data set) <b>for the sampled date in 2020</b>

In [79]:
print(yearly_best_sorted[1])
print(best_sites_codes[1])

('Los Angeles-North Main Street', ([3, 4], 13))
1103


In [80]:
# df = datafetcher.create_dataset(20200101, 20200102, site='1103', county='037', state='06', processed=True, verbose=False, vocs=True) # NOTE: This crashes because my kernel on my computer can't handle the computation.
# df.shape

### Include Emissions projections for chosen sight W/O VOC data

In [81]:
site_code = datafetcher.get_codes(LIST_SITES_BY_COUNTY, all=False, value='Los Angeles-North Main Street', nparams={'state':'06', 'county':'037'})

# Example of Site data using Los Angeles-North Main Street, Los Angeles, California
df = datafetcher.create_dataset(20180101, 20181231, site=site_code, county='037', state='06', processed=True, verbose=False, vocs=False)

/Users/tcintra/Desktop/research/chem150/chem150/venv/lib/python3.7/site-packages/pandas/core/frame.py:9166: FutureWarning: Passing 'suffixes' which cause duplicate columns {'latitude_x', 'longitude_x'} in the result is deprecated and will raise a MergeError in a future version.
  joined, frame, how=how, left_index=True, right_index=True


In [82]:
# NOTE: The (lat, lon) for LA North Main St is (34.07, -118.23)
# NOTE: The HEMCO CEDS netcdf files use bounding box every 50 km, so we want the closest box (34.25, -118.25)
print(df.shape)
df

(8760, 13)


,Carbon monoxide,Nitrogen dioxide (NO2),Ozone,PM2.5 - Local Conditions,Wind Direction - Resultant,Wind Speed - Resultant,Outdoor Temperature,Relative Humidity,Solar radiation,Ultraviolet radiation,Barometric pressure,latitude,longitude
datetime,,,,,,,,,,,,,
2018-01-01 00:00:00,1.4490,27.2,0.002,61.4,49.0,3.2,51.8,87.0,0.0,0.0,1009.0,34.06659,-118.22688
2018-01-01 01:00:00,1.5300,27.8,0.001,NaN,35.0,2.9,51.4,84.0,0.0,0.0,1009.0,34.06659,-118.22688
2018-01-01 02:00:00,1.4680,27.9,0.002,NaN,43.0,3.8,50.9,81.0,0.0,0.0,1009.0,34.06659,-118.22688
2018-01-01 03:00:00,1.4590,28.7,0.001,NaN,38.0,3.9,50.3,81.0,0.0,0.0,1009.0,34.06659,-118.22688
2018-01-01 04:00:00,1.4395,27.9,0.002,NaN,36.0,4.2,49.5,79.0,0.0,0.0,1009.0,34.06659,-118.22688
...,...,...,...,...,...,...,...,...,...,...,...,...,...
2018-12-31 19:00:00,0.6230,32.4,0.007,NaN,32.0,1.8,55.4,15.0,0.0,0.0,999.0,34.06659,-118.22688
2018-12-31 20:00:00,0.8230,34.0,0.003,NaN,25.0,2.0,52.4,19.0,0.0,0.0,1000.0,34.06659,-118.22688
2018-12-31 21:00:00,0.7425,31.7,0.005,NaN,30.0,2.3,51.7,18.0,0.0,0.0,1001.0,34.06659,-118.22688


In [86]:
nc_links, _ = datafetcher.get_ceds_links(year='2018')
la_lat = 34.25
la_lon = -118.25
ceds_df = datafetcher.make_ceds_df(la_lat, la_lon, nc_links)
ceds_df

,ALD2_agr,ALD2_ene,ALD2_ind,ALD2_rco,ALD2_shp,ALD2_slv,ALD2_tra,ALD2_wst,ALK4_butanes_agr,ALK4_butanes_ene,...,TOLU_tra,TOLU_wst,XYLE_agr,XYLE_ene,XYLE_ind,XYLE_rco,XYLE_shp,XYLE_slv,XYLE_tra,XYLE_wst
2018-01-01 00:00:00,0.0,1.268880e-15,3.655395e-14,4.180411e-13,0.0,0.0,1.599644e-13,1.295176e-13,0.0,1.349363e-12,...,1.650062e-12,3.429407e-13,0.0,2.333671e-13,1.001630e-11,1.231820e-13,1.085941e-14,4.498998e-11,1.357164e-12,2.165785e-13
2018-01-01 01:00:00,0.0,1.268880e-15,3.655395e-14,4.180411e-13,0.0,0.0,1.599644e-13,1.295176e-13,0.0,1.349363e-12,...,1.650062e-12,3.429407e-13,0.0,2.333671e-13,1.001630e-11,1.231820e-13,1.085941e-14,4.498998e-11,1.357164e-12,2.165785e-13
2018-01-01 02:00:00,0.0,1.268880e-15,3.655395e-14,4.180411e-13,0.0,0.0,1.599644e-13,1.295176e-13,0.0,1.349363e-12,...,1.650062e-12,3.429407e-13,0.0,2.333671e-13,1.001630e-11,1.231820e-13,1.085941e-14,4.498998e-11,1.357164e-12,2.165785e-13
2018-01-01 03:00:00,0.0,1.268880e-15,3.655395e-14,4.180411e-13,0.0,0.0,1.599644e-13,1.295176e-13,0.0,1.349363e-12,...,1.650062e-12,3.429407e-13,0.0,2.333671e-13,1.001630e-11,1.231820e-13,1.085941e-14,4.498998e-11,1.357164e-12,2.165785e-13
2018-01-01 04:00:00,0.0,1.268880e-15,3.655395e-14,4.180411e-13,0.0,0.0,1.599644e-13,1.295176e-13,0.0,1.349363e-12,...,1.650062e-12,3.429407e-13,0.0,2.333671e-13,1.001630e-11,1.231820e-13,1.085941e-14,4.498998e-11,1.357164e-12,2.165785e-13
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2018-12-31 19:00:00,0.0,1.270339e-15,3.655395e-14,4.400093e-13,0.0,0.0,1.599644e-13,1.295176e-13,0.0,1.350914e-12,...,1.650062e-12,3.429407e-13,0.0,2.336354e-13,1.001630e-11,1.296552e-13,1.147636e-14,4.498998e-11,1.357164e-12,2.165785e-13
2018-12-31 20:00:00,0.0,1.270339e-15,3.655395e-14,4.400093e-13,0.0,0.0,1.599644e-13,1.295176e-13,0.0,1.350914e-12,...,1.650062e-12,3.429407e-13,0.0,2.336354e-13,1.001630e-11,1.296552e-13,1.147636e-14,4.498998e-11,1.357164e-12,2.165785e-13
2018-12-31 21:00:00,0.0,1.270339e-15,3.655395e-14,4.400093e-13,0.0,0.0,1.599644e-13,1.295176e-13,0.0,1.350914e-12,...,1.650062e-12,3.429407e-13,0.0,2.336354e-13,1.001630e-11,1.296552e-13,1.147636e-14,4.498998e-11,1.357164e-12,2.165785e-13
2018-12-31 22:00:00,0.0,1.270339e-15,3.655395e-14,4.400093e-13,0.0,0.0,1.599644e-13,1.295176e-13,0.0,1.350914e-12,...,1.650062e-12,3.429407e-13,0.0,2.336354e-13,1.001630e-11,1.296552e-13,1.147636e-14,4.498998e-11,1.357164e-12,2.165785e-13


In [87]:
# Aggregate CEDS Emissions data
ceds_df_aggregated = datafetcher.aggregate_ceds_data(ceds_df)
print("Before Aggregating:", ceds_df.shape)
print("After Aggregating:", ceds_df_aggregated.shape)
ceds_df_aggregated.head(5)

Before Aggregating: (8760, 248)
After Aggregating: (8760, 31)


,ALD2,ALK4_butanes,ALK4_hexanes,ALK4_pentanes,BC,BENZ,BUTENE,C2H2,C2H4,C2H6,...,NH3,NO,OC,OTHER_AROM,OTHER_VOC,PRPE,SO2,TMB,TOLU,XYLE
2018-01-01 00:00:00,7.453460e-13,2.052799e-11,2.124349e-10,2.954461e-11,5.321832e-12,8.560323e-12,7.575214e-12,2.241559e-12,5.048348e-12,2.606391e-12,...,1.385000e-11,3.272592e-10,9.679659e-12,1.605214e-11,3.551276e-11,1.917994e-12,8.529435e-11,1.532717e-12,5.400424e-11,5.694743e-11
2018-01-01 01:00:00,7.453460e-13,2.052799e-11,2.124349e-10,2.954461e-11,5.321832e-12,8.560323e-12,7.575214e-12,2.241559e-12,5.048348e-12,2.606391e-12,...,1.385000e-11,3.272592e-10,9.679659e-12,1.605214e-11,3.551276e-11,1.917994e-12,8.529435e-11,1.532717e-12,5.400424e-11,5.694743e-11
2018-01-01 02:00:00,7.453460e-13,2.052799e-11,2.124349e-10,2.954461e-11,5.321832e-12,8.560323e-12,7.575214e-12,2.241559e-12,5.048348e-12,2.606391e-12,...,1.385000e-11,3.272592e-10,9.679659e-12,1.605214e-11,3.551276e-11,1.917994e-12,8.529435e-11,1.532717e-12,5.400424e-11,5.694743e-11
2018-01-01 03:00:00,7.453460e-13,2.052799e-11,2.124349e-10,2.954461e-11,5.321832e-12,8.560323e-12,7.575214e-12,2.241559e-12,5.048348e-12,2.606391e-12,...,1.385000e-11,3.272592e-10,9.679659e-12,1.605214e-11,3.551276e-11,1.917994e-12,8.529435e-11,1.532717e-12,5.400424e-11,5.694743e-11
2018-01-01 04:00:00,7.453460e-13,2.052799e-11,2.124349e-10,2.954461e-11,5.321832e-12,8.560323e-12,7.575214e-12,2.241559e-12,5.048348e-12,2.606391e-12,...,1.385000e-11,3.272592e-10,9.679659e-12,1.605214e-11,3.551276e-11,1.917994e-12,8.529435e-11,1.532717e-12,5.400424e-11,5.694743e-11


In [88]:
big_df = df.join(ceds_df_aggregated, how='inner')
print(big_df.shape)
big_df

(8760, 44)


,Carbon monoxide,Nitrogen dioxide (NO2),Ozone,PM2.5 - Local Conditions,Wind Direction - Resultant,Wind Speed - Resultant,Outdoor Temperature,Relative Humidity,Solar radiation,Ultraviolet radiation,...,NH3,NO,OC,OTHER_AROM,OTHER_VOC,PRPE,SO2,TMB,TOLU,XYLE
datetime,,,,,,,,,,,,,,,,,,,,,
2018-01-01 00:00:00,1.4490,27.2,0.002,61.4,49.0,3.2,51.8,87.0,0.0,0.0,...,1.385000e-11,3.272592e-10,9.679659e-12,1.605214e-11,3.551276e-11,1.917994e-12,8.529435e-11,1.532717e-12,5.400424e-11,5.694743e-11
2018-01-01 01:00:00,1.5300,27.8,0.001,NaN,35.0,2.9,51.4,84.0,0.0,0.0,...,1.385000e-11,3.272592e-10,9.679659e-12,1.605214e-11,3.551276e-11,1.917994e-12,8.529435e-11,1.532717e-12,5.400424e-11,5.694743e-11
2018-01-01 02:00:00,1.4680,27.9,0.002,NaN,43.0,3.8,50.9,81.0,0.0,0.0,...,1.385000e-11,3.272592e-10,9.679659e-12,1.605214e-11,3.551276e-11,1.917994e-12,8.529435e-11,1.532717e-12,5.400424e-11,5.694743e-11
2018-01-01 03:00:00,1.4590,28.7,0.001,NaN,38.0,3.9,50.3,81.0,0.0,0.0,...,1.385000e-11,3.272592e-10,9.679659e-12,1.605214e-11,3.551276e-11,1.917994e-12,8.529435e-11,1.532717e-12,5.400424e-11,5.694743e-11
2018-01-01 04:00:00,1.4395,27.9,0.002,NaN,36.0,4.2,49.5,79.0,0.0,0.0,...,1.385000e-11,3.272592e-10,9.679659e-12,1.605214e-11,3.551276e-11,1.917994e-12,8.529435e-11,1.532717e-12,5.400424e-11,5.694743e-11
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2018-12-31 19:00:00,0.6230,32.4,0.007,NaN,32.0,1.8,55.4,15.0,0.0,0.0,...,1.391520e-11,3.342732e-10,9.898709e-12,1.605710e-11,3.551342e-11,1.957491e-12,8.600769e-11,1.533309e-12,5.402841e-11,5.695479e-11
2018-12-31 20:00:00,0.8230,34.0,0.003,NaN,25.0,2.0,52.4,19.0,0.0,0.0,...,1.391520e-11,3.342732e-10,9.898709e-12,1.605710e-11,3.551342e-11,1.957491e-12,8.600769e-11,1.533309e-12,5.402841e-11,5.695479e-11
2018-12-31 21:00:00,0.7425,31.7,0.005,NaN,30.0,2.3,51.7,18.0,0.0,0.0,...,1.391520e-11,3.342732e-10,9.898709e-12,1.605710e-11,3.551342e-11,1.957491e-12,8.600769e-11,1.533309e-12,5.402841e-11,5.695479e-11


## Compare CEDS Emissions data to PAMS_VOC Data

Which compounds appear in both datasets? What are the differences between the datasets.

It seeems that AQS reports verry granular data for speecific compounds, whereas CEDS providees mor generalized emissions data. For example, CEDS reports emissions for 'butanes' whereas AQS reports '2,2-dimethylbutane' instead. Furthermore, CEDS has more opaque naming conventions, like 'other alkanals' which might encompass any aldehydes in the AQS dataset. I am not sure how to match emissions to AQS VOC compounds without manually going through each compound and understanding the necessary chemistry.

For now: simply keep the ones that match in name exactly.

In [149]:
# NOTE: There are 60 PAMS_VOCS and LA North Main St has data on all of them:
print(f"Site has all PAMS_VOCS data: {len(voc_r['Metadata']['codes']) == len([x[1] for x in voc_r['Data']['Los Angeles-North Main Street'] if x[1] == True])}\n")

vocs = sorted([datafetcher.find_name(code) for code in voc_r['Metadata']['codes']])
print(f"AQS Compound Names:\n{vocs}\n")

print(f"CEDS Emissions Names:\n{datafetcher.ceds_compounds}\n")

Site has all PAMS_VOCS data: True

AQS Compound Names:
['1,2,3-Trimethylbenzene', '1,2,4-Trimethylbenzene', '1,3,5-Trimethylbenzene', '1-Butene', '1-Pentene', '2,2,4-Trimethylpentane', '2,2-Dimethylbutane', '2,3,4-Trimethylpentane', '2,3-Dimethylbutane', '2,3-Dimethylpentane', '2,4-Dimethylpentane', '2-Methylheptane', '2-Methylhexane', '2-Methylpentane', '3-Methylheptane', '3-Methylhexane', '3-Methylpentane', 'Acetaldehyde', 'Acetone', 'Acetylene', 'Benzene', 'Cyclohexane', 'Cyclopentane', 'Ethane', 'Ethylbenzene', 'Ethylene', 'Formaldehyde', 'Isobutane', 'Isopentane', 'Isoprene', 'Isopropylbenzene', 'Methylcyclohexane', 'Methylcyclopentane', 'Propane', 'Propylene', 'Styrene', 'Sum of PAMS target compounds', 'Toluene', 'Total NMOC (non-methane organic compound)', 'cis-2-Butene', 'cis-2-Pentene', 'm-Diethylbenzene', 'm-Ethyltoluene', 'm/p Xylene', 'n-Butane', 'n-Decane', 'n-Heptane', 'n-Hexane', 'n-Nonane', 'n-Octane', 'n-Pentane', 'n-Propylbenzene', 'n-Undecane', 'o-Ethyltoluene', 'o-X

## Match CEDS emissions data to AQS measurements

Using CEDS wiki page, match CEDS categories to the compounds in the AQS dataframe

In [150]:
# Compounds present in CEDS data
print(datafetcher.ceds_compounds)

{'ALD2': 'other_alkanals', 'ALK4_butanes': 'butanes', 'ALK4_hexanes': 'hexanes_plus_higher_alkanes', 'ALK4_pentanes': 'pentanes', 'BC': None, 'BENZ': 'benzene', 'BUTENE': 'other_alkenes_and_alkynes', 'C2H2': 'ethyne', 'C2H4': 'ethene', 'C2H6': 'ethane', 'C3H8': 'propane', 'CH2O': 'methanal', 'CHC': 'chlorinated_hydrocarbons', 'CO': None, 'CO2': None, 'EOH': 'alcohols', 'ESTERS': 'esters', 'ETHERS': 'ethers', 'HCOOH': 'acids', 'MEK': 'ketones', 'N2O': None, 'NH3': None, 'NO': None, 'OC': None, 'OTHER_AROM': 'other_aromatics', 'OTHER_VOC': 'other_voc', 'PRPE': 'propene', 'SO2': None, 'TMB': 'trimethylbenzenes', 'TOLU': 'toluene', 'XYLE': 'xylene'}


In [161]:
# Create JSON map of CEDS species names above and which AQS compounds have those emissions

ceds_aqs_map = {
    'ALD2' : {'verify' : False, 'matches' : ['Acetaldehyde'], 'notes' : ''},
    'ALK4_butanes' : {'verify' : True, 'matches' : ['2,2-Dimethylbutane', '2,3-Dimethylbutane', 'Isobutane', 'n-Butane'], 'notes' : 'ALK4_butanes has the attribute name: butanes so I believe this is emissions data for all butanes. What do they mean by C4 alkanes?'},
    'ALK4_hexanes' : {'verify' : True, 'matches' : ['2-Methylhexane', '3-Methylhexane', 'Cyclohexane', 'Methylcyclohexane', 'n-Hexane'], 'notes' : 'Same as ALK4_butanes'},
    'ALK4_pentanes' : {'verify' : True, 'matches' : ['2,2,4-Trimethylpentane', '2,3,4-Trimethylpentane', '2,3-Dimethylpentane', '2,4-Dimethylpentane', '2-Methylpentane', '3-Methylpentane', 'Cyclopentane', 'Isopentane', 'Methylcyclopentane', 'n-Pentane'], 'notes' : 'Same as ALK4_butanes'},
    'BC' : {'verify' : True, 'matches' : [], 'notes' : 'I am not sure what BC is, it does not have an entry in the wiki page but I think it stands for black carbon.'},
    'BENZ' : {'verify' : True, 'matches' : ['Benzene', 'Ethylbenzene', 'Isopropylbenzene', 'm-Diethylbenzene', 'n-Propylbenzene', 'p-Diethylbenzene'], 'notes' : 'Some of these are obvious but I am not sure if all benzene derivatives fall under this category.'},
    'BUTENE' : {'verify' : True, 'matches' : ['1-Butene', 'cis-2-Butene', 'trans-2-Butene', '1-Pentene'], 'notes' : 'These stand for other_alkenes_and_alkynes, not sure if any of the compounds fall under this category.'},
    'C2H2' : {'verify' : False, 'matches' : ['Acetylene'], 'notes' : 'Ethyne and acetylene are the same thing.'},
    'C2H4' : {'verify' : False, 'matches' : ['Ethylene'], 'notes' : ''},
    'C2H6' : {'verify' : False, 'matches' : ['Ethane'], 'notes' : ''},
    'C3H8' : {'verify' : False, 'matches' : ['Propane'], 'notes' : ''},
    'CH2O' : {'verify' : False, 'matches' : ['Formaldehyde'], 'notes' : ''},
    'CHC' : {'verify' : True, 'matches' : [], 'notes' : 'Are any of the AQS compounds chlorinated hydrocarbons?'},
    'CO' : {'verify' : False, 'matches' : [], 'notes' : ''},
    'CO2' : {'verify' : False, 'matches' : [], 'notes' : ''},
    'EOH' : {'verify' : True, 'matches' : [], 'notes' : 'I am not sure if any of these are alcohols.'},
    'ESTERS' : {'verify' : True, 'matches' : [], 'notes' : 'I am not sure if any of these are esters.'},
    'ETHERS' : {'verify' : True, 'matches' : [], 'notes' : 'I am not sure if any of these are ethers.'},
    'HCOOH' : {'verify' : True, 'matches' : [], 'notes' : 'I am not sure if any of these are formic acids.'},
    'MEK' : {'verify' : True, 'matches' : [], 'notes' : 'I am not sure if any of these are methyl ethyl ketones.'},
    'N2O' : {'verify' : True, 'matches' : [], 'notes' : 'I am not sure if any of these are nitrous oxides.'},
    'NH3' : {'verify' : True, 'matches' : [], 'notes' : 'I am not sure if any of these are ammonia.'},
    'NO' : {'verify' : True, 'matches' : [], 'notes' : 'I am not sure if any of these are nitric oxide.'},
    'OC' : {'verify' : True, 'matches' : [], 'notes' : 'What does this stand for? It is not explicit in wiki page.'},
    'OTHER_AROM' : {'verify' : True, 'matches' : [], 'notes': 'I think this is where some of the compounds under BENZ should actually be?'},
    'OTHER_VOC' : {'verify' : True, 'matches' : [], 'notes' : 'What would fall under this category?'},
    'PRPE' : {'verify' : False, 'matches' : ['Propylene'], 'notes' : ''},
    'SO2' : {'verify' : False, 'matches' : [], 'notes' : 'No sulfur dioxide.'},
    'TMB' : {'verify' : False, 'matches' : ['1,2,3-Trimethylbenzene', '1,2,4-Trimethylbenzene', '1,3,5-Trimethylbenzene'], 'notes' : ''},
    'TOLU' : {'verify' : True, 'matches' : ['Toluene', 'm-Ethyltoluene', 'o-Ethyltoluene', 'p-Ethyltoluene'], 'notes' : 'Are these all under toluene?'},
    'XYLE' : {'verify' : True, 'matches' : ['m/p Xylene', 'o-Xylene'], 'notes' : 'Are these all under xylene?'}
}

In [162]:
matched = []
for k in ceds_aqs_map:
    matched += ceds_aqs_map[k]['matches']
unmatched = [x for x in vocs if x not in matched]
unmatched

['2-Methylheptane',
 '3-Methylheptane',
 'Acetone',
 'Isoprene',
 'Styrene',
 'Sum of PAMS target compounds',
 'Total NMOC (non-methane organic compound)',
 'cis-2-Pentene',
 'n-Decane',
 'n-Heptane',
 'n-Nonane',
 'n-Octane',
 'n-Undecane',
 'trans-2-Pentene']

### NOTES ON UNMATCHED AQS COMPOUNDS

* Do we drop the Sum of PAMS target compounds?
* What are non-methane organic compounds? Is that anything without a methane group?
* Are all the remaining compounds just under a broader category in CEDS that I missed?

In [164]:
len(matched) + len(unmatched) == len(vocs)

True

In [165]:
print(f"Matched {len(matched)} out of {len(vocs)} compounds")

Matched 45 out of 59 compounds


### Look at which matches are more ambiguous

In [172]:
pd.DataFrame(ceds_aqs_map).T

,verify,matches,notes
ALD2,False,[Acetaldehyde],
ALK4_butanes,True,"[2,2-Dimethylbutane, 2,3-Dimethylbutane, Isobu...",ALK4_butanes has the attribute name: butanes s...
ALK4_hexanes,True,"[2-Methylhexane, 3-Methylhexane, Cyclohexane, ...",Same as ALK4_butanes
ALK4_pentanes,True,"[2,2,4-Trimethylpentane, 2,3,4-Trimethylpentan...",Same as ALK4_butanes
BC,True,[],"I am not sure what BC is, it does not have an ..."
BENZ,True,"[Benzene, Ethylbenzene, Isopropylbenzene, m-Di...",Some of these are obvious but I am not sure if...
BUTENE,True,"[1-Butene, cis-2-Butene, trans-2-Butene, 1-Pen...","These stand for other_alkenes_and_alkynes, not..."
C2H2,False,[Acetylene],Ethyne and acetylene are the same thing.
C2H4,False,[Ethylene],
C2H6,False,[Ethane],


In [171]:
ambiguous = {k : v for k,v in ceds_aqs_map.items() if v['verify'] == True}
pd.DataFrame(ambiguous).T

,verify,matches,notes
ALK4_butanes,True,"[2,2-Dimethylbutane, 2,3-Dimethylbutane, Isobu...",ALK4_butanes has the attribute name: butanes s...
ALK4_hexanes,True,"[2-Methylhexane, 3-Methylhexane, Cyclohexane, ...",Same as ALK4_butanes
ALK4_pentanes,True,"[2,2,4-Trimethylpentane, 2,3,4-Trimethylpentan...",Same as ALK4_butanes
BC,True,[],"I am not sure what BC is, it does not have an ..."
BENZ,True,"[Benzene, Ethylbenzene, Isopropylbenzene, m-Di...",Some of these are obvious but I am not sure if...
BUTENE,True,"[1-Butene, cis-2-Butene, trans-2-Butene, 1-Pen...","These stand for other_alkenes_and_alkynes, not..."
CHC,True,[],Are any of the AQS compounds chlorinated hydro...
EOH,True,[],I am not sure if any of these are alcohols.
ESTERS,True,[],I am not sure if any of these are esters.
ETHERS,True,[],I am not sure if any of these are ethers.


### Join CRITERIA, MET, PAMS_VOCS, and Emissions data into one df

In [222]:
# TODO: These create a lot of missing values, which we need to take care of by interpolating or something
# NOTE: Some VOC data has multiple collection methods. For now I am only taking the first set, meaening thee measurements of the first method
# TODO: Choose a method, take the average?
matched_vocs_df = datafetcher.get_voc_data(20180101, 20181231, '06', '037', '1103', matched)
matched_vocs_df

,Benzene,Ethane,Propane,Toluene
datetime,,,,
2018-01-02 00:00:00,3.72,88.66,32.69,12.46
2018-01-02 01:00:00,1.74,44.05,18.08,7.49
2018-01-02 02:00:00,1.74,44.05,18.08,7.49
2018-01-02 03:00:00,1.74,44.05,18.08,7.49
2018-01-02 04:00:00,1.74,44.05,18.08,7.49
...,...,...,...,...
2018-12-27 20:00:00,1.32,NaN,NaN,2.66
2018-12-27 21:00:00,1.32,NaN,NaN,2.66
2018-12-27 22:00:00,1.32,NaN,NaN,2.66


In [223]:
# Note: What should the columns name be? Idea: Ethane_AQS, Ethane_CEDS? Or better naming convention? Should I drop emissions data that doesn't have corresponding AQS VOC?
big_df.join(matched_vocs_df, how='inner')

,Carbon monoxide,Nitrogen dioxide (NO2),Ozone,PM2.5 - Local Conditions,Wind Direction - Resultant,Wind Speed - Resultant,Outdoor Temperature,Relative Humidity,Solar radiation,Ultraviolet radiation,...,OTHER_VOC,PRPE,SO2,TMB,TOLU,XYLE,Benzene,Ethane,Propane,Toluene
datetime,,,,,,,,,,,,,,,,,,,,,
2018-01-02 00:00:00,1.2580,37.7,0.001,42.35,28.0,2.6,54.1,74.0,0.0,0.0,...,3.551276e-11,1.917994e-12,8.529435e-11,1.532717e-12,5.400424e-11,5.694743e-11,3.72,88.66,32.69,12.46
2018-01-02 01:00:00,1.2645,36.8,0.001,NaN,38.0,2.4,53.9,74.0,0.0,0.0,...,3.551276e-11,1.917994e-12,8.529435e-11,1.532717e-12,5.400424e-11,5.694743e-11,1.74,44.05,18.08,7.49
2018-01-02 02:00:00,1.1830,36.2,0.001,NaN,36.0,4.1,53.9,72.0,0.0,0.0,...,3.551276e-11,1.917994e-12,8.529435e-11,1.532717e-12,5.400424e-11,5.694743e-11,1.74,44.05,18.08,7.49
2018-01-02 03:00:00,1.1510,35.5,0.001,NaN,16.0,1.7,54.2,71.0,0.0,0.0,...,3.551276e-11,1.917994e-12,8.529435e-11,1.532717e-12,5.400424e-11,5.694743e-11,1.74,44.05,18.08,7.49
2018-01-02 04:00:00,1.2660,35.1,0.001,NaN,24.0,2.2,54.1,74.0,0.0,0.0,...,3.551276e-11,1.917994e-12,8.529435e-11,1.532717e-12,5.400424e-11,5.694743e-11,1.74,44.05,18.08,7.49
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2018-12-27 20:00:00,0.2130,5.0,0.031,NaN,345.0,13.1,56.6,26.0,0.0,0.0,...,3.551342e-11,1.957491e-12,8.600769e-11,1.533309e-12,5.402841e-11,5.695479e-11,1.32,NaN,NaN,2.66
2018-12-27 21:00:00,0.2105,5.8,0.031,NaN,352.0,9.1,56.6,21.0,0.0,0.0,...,3.551342e-11,1.957491e-12,8.600769e-11,1.533309e-12,5.402841e-11,5.695479e-11,1.32,NaN,NaN,2.66
2018-12-27 22:00:00,0.2200,7.6,0.032,NaN,358.0,8.3,56.9,15.0,0.0,0.0,...,3.551342e-11,1.957491e-12,8.600769e-11,1.533309e-12,5.402841e-11,5.695479e-11,1.32,NaN,NaN,2.66


In [229]:
# PM2.5 Missing values
big_df['PM2.5 - Local Conditions'].isna().sum()

8413